<a href="https://colab.research.google.com/github/ericyoc/quantum-circuits/blob/main/quantum_circuit_cirq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cirq (Google)

In [3]:
#!pip install cirq
import cirq
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Set GPU memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
def create_cirq_circuit():
    # Load MNIST dataset
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()

    # Normalize the input data
    train_data = train_data / 255.0
    test_data = test_data / 255.0

    # Reshape the input data
    train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
    test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

    # Create a quantum circuit for image classification
    qubits = cirq.GridQubit.rect(2, 2)
    circuit = cirq.Circuit()

    # Apply quantum operations to the qubits
    circuit.append(cirq.H(qubits[0]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))
    circuit.append(cirq.CNOT(qubits[1], qubits[2]))
    circuit.append(cirq.CNOT(qubits[2], qubits[3]))

    # Create a classical model
    with tf.device('/GPU:0'):  # Use GPU for model creation and training
        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                      metrics=['accuracy'])

        # Train the model
        model.fit(train_data, train_labels, epochs=10, batch_size=32)

    # Get the circuit diagram
    circuit_diagram = circuit.to_text_diagram()

    return model, test_data, circuit_diagram

In [6]:
# Call the function to create the model and get the circuit diagram
model, test_data, diagram = create_cirq_circuit()

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 1.2668 - accuracy: 0.5931
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3367 - accuracy: 0.5325
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 1.4081 - accuracy: 0.5069
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.3981 - accuracy: 0.4900
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.4072 - accuracy: 0.4898
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 1.5131 - accuracy: 0.4522
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 1.4880 - accuracy: 0.4525
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 1.4292 - accuracy: 0.4785
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 1.4318 - accuracy: 0.4719
Epoch 10/10
1875/

In [7]:
# Predict the digit for a test image
test_image = test_data[0]
predicted_digit = model.predict(test_image[np.newaxis, ...])
predicted_digit = np.argmax(predicted_digit)
print(f"Predicted digit: {predicted_digit}")

1/1 [==============================] - 0s 91ms/step
Predicted digit: 7


In [8]:
print("Circuit Diagram:")
print(diagram)

Circuit Diagram:
(0, 0): ───H───@───────────
               │
(0, 1): ───────X───@───────
                   │
(1, 0): ───────────X───@───
                       │
(1, 1): ───────────────X───
